# 신경망 모델 구성하기
신경망은 데이터에 대한 연산을 수행하는 계층 / 모듈로 구성되어 있다.  
torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다.  
Pytorch의 모든 모듈은 nn.Module의 하위 클래스이다.  
신경망은 다른 모듈로 구성된 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.


In [4]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기
가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 한다.  
torch.cuda 또는 torch.backends.mps가 사용 가능한지 확인해보고, 그렇지 않으면 CPU를 계속 사용한다.

In [5]:
device = (
    'cuda' 
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f'Using {device} device')

Using mps device


## 클래스 정의하기
신경망 모델을 nn.Module 의 하위클래스로 정의하고, __init__에서 신경망 계층들을 초기화한다.  
nn.Module 을 상속받은 모델 클래스는 forward 메소드에 입력 데이터에 연산들을 구현한다.

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork의 인스턴스를 생성하고 이를 device로 이동한 뒤, 구조를 출력한다.

In [11]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달한다. 이는 일부 백그라운드 연산들과 함께 모델의 forward를 실행.  model.forward()를 직접 호출하지 마세요!

모델에 입력을 전달하는 호출하면 2차원 텐서를 반환한다. 2차원 텐서의 dim = 0은 각 분류에 대해 원시 예측값 10개가, dim = 1에는 각 출력의 개별값들이 해당한다. 원시 예측값은 nn.Softmax 모듈의 인스턴스에 통과시켜 예측확률을 얻는다.


In [12]:
X = torch.rand(1,28,28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([-9223372036854775808], device='mps:0')


# 모델 계층(Layer)
FashionMNIST 모델의 계층을 살펴보자. 이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보자.

In [15]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten
nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784픽셀 값을 갖는 연속된 배열로 반환한다. 
(dim = 0의 미니배치 차원은 유지된다)

In [17]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear
선형 계층은 저장도니 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈

In [18]:
layer1 = nn.Linear(in_features = 28*28, out_features =20)
hidden1 = layer1(flat_image)
print(hidden1.size())


torch.Size([3, 20])


## nn.ReLU
비선형 활성화는 모델의 입력과 출력 사이에 복잡한 관계를 만든다.   
비선형 활성화는 선형 변환 후에 적용되어 비선형성을 도입하고, 신경망이 다야한 현상을 학습할 수 있도록 돕는다.

이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수 있다.

In [20]:
print(f'Before ReLU : {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU : {hidden1}')

Before ReLU : tensor([[0.1310, 0.0000, 0.1783, 0.0000, 0.5375, 0.0000, 0.0000, 0.5331, 0.0740,
         0.0000, 0.0157, 0.0915, 0.1489, 0.1417, 0.0178, 0.0000, 0.0000, 0.0000,
         0.1858, 0.1405],
        [0.1612, 0.0484, 0.0000, 0.2558, 0.6876, 0.0000, 0.0107, 0.2107, 0.0000,
         0.0722, 0.0000, 0.1861, 0.0000, 0.0454, 0.0172, 0.0000, 0.0000, 0.0000,
         0.4417, 0.0009],
        [0.0000, 0.1465, 0.1976, 0.0000, 0.0923, 0.0000, 0.0000, 0.1230, 0.0000,
         0.0000, 0.0000, 0.0803, 0.0000, 0.0846, 0.1642, 0.0000, 0.0000, 0.0000,
         0.1930, 0.3775]], grad_fn=<ReluBackward0>)


After ReLU : tensor([[0.1310, 0.0000, 0.1783, 0.0000, 0.5375, 0.0000, 0.0000, 0.5331, 0.0740,
         0.0000, 0.0157, 0.0915, 0.1489, 0.1417, 0.0178, 0.0000, 0.0000, 0.0000,
         0.1858, 0.1405],
        [0.1612, 0.0484, 0.0000, 0.2558, 0.6876, 0.0000, 0.0107, 0.2107, 0.0000,
         0.0722, 0.0000, 0.1861, 0.0000, 0.0454, 0.0172, 0.0000, 0.0000, 0.0000,
         0.4417, 0.0009],
     

## nn.Sequential
nn.Sequential은 순서를 갖는 모듈의 컨테이너이다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다. 순차 컨테이너를 사용하여 아래의 seq_models 와 같은 신경망을 빠르게 만들 수 있다.

In [22]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax
신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달된 ([-infinity, infinity] 범위의 원시 값인) logits를 반환한다. logits는 모델의 각 분류에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정된다. dim 매개변수는 값의 합이 1이 되는 차원을 나타낸다.

In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 모델 매개변수
신경망 내부의 많은 계층들은 매개변수화 된다. 즉, 학습 중에 최적화되는 가중치와 편향을 연결지어진다. nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동을 추적되며, 모델의 parameter() named_parameter() 메소드로 모든 매개변수의 크기와 값을 출력한다.

In [28]:
print(f'Module structure : {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer : {name} | Size : {param.size()} | Values : {param[:2]}\n')

Module structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[ 0.0297, -0.0124, -0.0224,  ..., -0.0187, -0.0170, -0.0049],
        [-0.0351, -0.0277, -0.0118,  ...,  0.0033, -0.0206,  0.0035]],
       device='mps:0', grad_fn=<SliceBackward0>)

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0168,  0.0293], device='mps:0', grad_fn=<SliceBackward0>)

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0060, -0.0081, -0.0080,  ..., -0.0191, -0.0170,  0.0172],
        [ 0.0200,  0.0339, -0.0143,  ...,  0.0320, -0.0301, -0.0099]],
       device='mps:0', grad_fn=

In [29]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out

tensor([[4., 1., 1., 1.],
        [1., 4., 1., 1.],
        [1., 1., 4., 1.],
        [1., 1., 1., 4.],
        [1., 1., 1., 1.]], grad_fn=<TBackward0>)